In [1]:
from basicsr.archs.basicvsr_arch import BasicVSR
import torch
import numpy as np
from glob import glob
import cv2
from torchviz import make_dot
from torchsummary import summary
from torch.utils.tensorboard import SummaryWriter

C:\Users\Bijin\Anaconda3\envs\finalsem\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
model = BasicVSR(num_feat=64, num_block=30)
model.load_state_dict(torch.load("./BasicVSR_REDS4-543c8261.pth")['params'], strict=True)

<All keys matched successfully>

In [3]:
model.to('cuda')
_ = model.eval()

In [4]:
frames = glob("video/*.jpg")
frames = sorted(frames)

In [5]:
inputs = []
for frame in frames[:5]:
    img = cv2.imread(frame)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    size = img.shape[:2]
    img = cv2.resize(img, (size[1]//2, size[0]//2))
    inputs.append(img)

In [6]:
inputs = np.array(inputs).astype(np.float32)
inputs /= 255.0

In [7]:
inputs.shape

(5, 72, 128, 3)

In [8]:
inputs = np.transpose(inputs, (0, 3, 1, 2))

In [9]:
inputs = torch.tensor(inputs).unsqueeze(0).to("cuda")

In [10]:
inputs.shape

torch.Size([1, 5, 3, 72, 128])

In [11]:
summary(model, input_size=inputs.shape[1:])

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1             [-1, 32, 3, 4]          12,576
              ReLU-2             [-1, 32, 3, 4]               0
            Conv2d-3             [-1, 64, 3, 4]         100,416
              ReLU-4             [-1, 64, 3, 4]               0
            Conv2d-5             [-1, 32, 3, 4]         100,384
              ReLU-6             [-1, 32, 3, 4]               0
            Conv2d-7             [-1, 16, 3, 4]          25,104
              ReLU-8             [-1, 16, 3, 4]               0
            Conv2d-9              [-1, 2, 3, 4]           1,570
      BasicModule-10              [-1, 2, 3, 4]               0
           Conv2d-11             [-1, 32, 6, 8]          12,576
             ReLU-12             [-1, 32, 6, 8]               0
           Conv2d-13             [-1, 64, 6, 8]         100,416
             ReLU-14             [-1, 6

In [13]:
# with torch.no_grad():
outputs = model(inputs[:,:10])

In [12]:
outputs = outputs.to('cpu')
model = model.cpu()
make_dot(outputs.mean(), params=dict(model.named_parameters()))

In [55]:
outputs = outputs.squeeze(0).cpu().numpy()

In [56]:
outputs = np.transpose(outputs, (0, 2, 3, 1))

In [57]:
# os.mkdir("out")
for i, out in enumerate(outputs):
    out = out * 255
    out = out.astype(np.uint8)
    out = cv2.cvtColor(out, cv2.COLOR_RGB2BGR)
    cv2.imwrite(f"out/out_{i}.jpg", out)

In [13]:
# model_traced.cuda()
outputs = model_traced(inputs[:, 10:])

RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
RuntimeError: The following operation failed in the TorchScript interpreter.
Traceback of TorchScript (most recent call last):
RuntimeError: CUDA out of memory. Tried to allocate 20.00 MiB (GPU 0; 6.00 GiB total capacity; 5.01 GiB already allocated; 0 bytes free; 5.38 GiB reserved in total by PyTorch)



In [54]:
import cv2 
import numpy as np
from PIL import Image
import torch
from torch.nn.functional import interpolate
from glob import glob

In [57]:
imgs = glob("video/*.jpg")
imgs = [Image.open(im) for im in imgs]

In [61]:
Image.open(imgs[0])

AttributeError: module 'PIL.Image' has no attribute 'read'

In [58]:
imgs = np.array(imgs).astype(np.float32)
imgs /= 255.0
imgs = np.transpose(imgs, (0, 2, 3, 1)) 

C:\Users\Bijin\Anaconda3\envs\finalsem\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The input object of type 'JpegImageFile' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'JpegImageFile', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  """Entry point for launching an IPython kernel.
C:\Users\Bijin\Anaconda3\envs\finalsem\lib\site-packages\ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


TypeError: float() argument must be a string or a number, not 'JpegImageFile'

In [1]:
from basic_vsr import BasicVSR
from spynet import SpyNet
from ifnet import IFNet

import torch
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

c:\Users\Bijin\Anaconda3\envs\finalsem\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
spynet = SpyNet().to('cuda')
ifnet = IFNet().to('cuda')

basic_if = BasicVSR(ifnet).to('cuda')
basic_spy = BasicVSR(spynet).to('cuda')

In [3]:
x1 = torch.zeros((1, 3, 256, 256)).to('cuda')
x2 = torch.ones((1, 3, 256, 256)).to('cuda')
x3 = torch.randn((1, 3, 256, 256)).to('cuda')

In [4]:
x = torch.cat((x1, x2, x3), 0).unsqueeze(0)
x.shape

torch.Size([1, 3, 3, 256, 256])

In [5]:
basic_if(x).shape

c:\Users\Bijin\Anaconda3\envs\finalsem\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


torch.Size([1, 3, 3, 1024, 1024])

In [6]:
basic_spy(x).shape

torch.Size([1, 3, 3, 1024, 1024])

In [9]:
x1_new = x[:, :-3].view(-1, 3, 256, 256)
x2_new = x[:, 3:].view(-1, 3, 256, 256)

In [9]:
a = x.view(1, 3, 3, 256, 256)
torch.all(a[0][0] == x1[0])

tensor(True, device='cuda:0')

In [10]:
print(x1_new.shape)
print(x2_new.shape)

torch.Size([2, 3, 256, 256])
torch.Size([2, 3, 256, 256])


In [11]:
assert torch.all(x1_new[0] == x1[0])
assert torch.all(x1_new[1] == x2[0])
assert torch.all(x1_new[1] == x2_new[0])
assert torch.all(x2_new[1] == x3[0])

In [12]:
b, c, h, w = x.size()

In [13]:
if_flow = basic_if.get_flow(x)

c:\Users\Bijin\Anaconda3\envs\finalsem\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [14]:
spy_flow = basic_spy.get_flow(x)

In [15]:
flow_list, mask, merged, flow_teacher, merged_teacher, loss_distill = basic_if.flownet(x1, x2, x3, flow_only=False)

In [1]:
from generator import BasicGenerator
import torch

c:\Users\Bijin\Anaconda3\envs\finalsem\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
gen = BasicGenerator("models/spynet_sintel_final-3d2a1287.pth")

In [3]:
gen.to('cuda')
x = torch.randn((2, 4, 3, 64, 64), device='cuda')

In [4]:
gen(x).shape

c:\Users\Bijin\Anaconda3\envs\finalsem\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


torch.Size([2, 7, 3, 256, 256])

In [6]:
a = torch.full((1, 4, 3, 2, 2), 1)
b = torch.full((1, 3, 3, 2, 2), 2)

c = torch.zeros((1, 7, 3, 2, 2))
c[:, ::2] = a
c[:, 1::2] = b
c[:, :3]

tensor([[[[[1., 1.],
           [1., 1.]],

          [[1., 1.],
           [1., 1.]],

          [[1., 1.],
           [1., 1.]]],


         [[[2., 2.],
           [2., 2.]],

          [[2., 2.],
           [2., 2.]],

          [[2., 2.],
           [2., 2.]]],


         [[[1., 1.],
           [1., 1.]],

          [[1., 1.],
           [1., 1.]],

          [[1., 1.],
           [1., 1.]]]]])

In [1]:
from model_utils import BicubicUpsampler
import torch
from ifnet import IFNet
from basic_vsr import BasicVSR
from model_utils import warp

c:\Users\Bijin\Anaconda3\envs\finalsem\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
bic = BicubicUpsampler(4).to('cuda')
ifnet = IFNet().to('cuda')
basic = BasicVSR(ifnet, num_block=30, num_feat=64).to('cuda')

In [3]:
a = torch.randn((2, 4, 3, 64, 64), device='cuda')
b, n, c, h, w = a.size()
a1 = a[:, :-1].reshape(-1, 3, 64, 64)
a2 = a[:, 1:].reshape(-1, 3, 64, 64)

In [4]:
flows, mask, _, _, _, _ = ifnet(a1, a2, flow_only=False)
flows = flows[2]
# flows[0].shape

c:\Users\Bijin\Anaconda3\envs\finalsem\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [5]:
upscaled_flows = bic(flows)
upscaled_masks = bic(mask)

In [6]:
upscaled_flows.shape

torch.Size([6, 4, 256, 256])

In [7]:
flows_forward = flows[:, :2].view(b, n - 1, 2, h, w)
flows_backward = flows[:, 2:4].view(b, n - 1, 2, h, w)
upscaled = basic(a, flows_forward, flows_backward)

In [8]:
upscaled_a1 = upscaled[:, :-1].reshape(-1, 3, 256, 256)
upscaled_a2 = upscaled[:, 1:].reshape(-1, 3, 256, 256)

In [9]:
upscaled_warp1 = warp(upscaled_a1, upscaled_flows[:, :2].permute(0, 2, 3, 1)) 
upscaled_warp2 = warp(upscaled_a2, upscaled_flows[:, 2:4].permute(0, 2, 3, 1)) 

In [10]:
upscaled_warp1.shape

torch.Size([6, 3, 256, 256])

In [14]:
upscaled_merge = upscaled_warp1 * upscaled_masks + upscaled_warp2 * (1-upscaled_masks)
upscaled_merge = upscaled_merge.reshape(2, 3, 3, 256, 256)

In [15]:
mixed = torch.zeros((2, 7, 3, 256, 256))
mixed[:, ::2] = upscaled
mixed[:, 1::2] = upscaled_merge

In [1]:
from generator import UpscalingGenerator
import torch

c:\Users\Bijin\Anaconda3\envs\finalsem\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
up_gen = UpscalingGenerator().to('cuda')
a = torch.randn((2, 4, 3, 64, 64), device='cuda')

In [4]:
up_gen(a).shape

torch.Size([2, 7, 3, 256, 256])

In [1]:
from generator import BasicToRifeGenerator, RifeToBasicGenerator, UpscalingGenerator
import torch

c:\Users\Bijin\Anaconda3\envs\finalsem\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
b2r = BasicToRifeGenerator("models/spynet_sintel_final-3d2a1287.pth").to('cuda')
r2b = RifeToBasicGenerator("models/spynet_sintel_final-3d2a1287.pth").to('cuda')
up_gen = UpscalingGenerator()

In [3]:
a = torch.randn((1, 4, 3, 64, 64), device='cuda')

In [4]:
r2b(a)

c:\Users\Bijin\Anaconda3\envs\finalsem\lib\site-packages\torch\functional.py:568: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\TensorShape.cpp:2228.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


tensor([[[[[ 5.0632e-02,  5.9749e-02, -3.7985e-02,  ..., -9.4939e-03,
            -5.1488e-02, -5.8322e-02],
           [ 5.1178e-02,  5.6506e-02, -4.2660e-02,  ..., -1.5297e-02,
            -6.3618e-02, -5.7294e-02],
           [ 1.3179e-01,  1.3670e-01,  1.9909e-02,  ..., -1.6341e-01,
            -2.1384e-01, -2.1166e-01],
           ...,
           [-2.4926e+00, -2.4867e+00, -2.1888e+00,  ...,  8.1997e-01,
             7.5529e-01,  7.5679e-01],
           [-2.8034e+00, -2.7997e+00, -2.4642e+00,  ...,  9.7682e-01,
             8.6663e-01,  8.7124e-01],
           [-2.7973e+00, -2.7962e+00, -2.4612e+00,  ...,  9.7526e-01,
             8.6630e-01,  8.6575e-01]],

          [[-9.2775e-01, -9.1932e-01, -8.2868e-01,  ..., -1.3328e-01,
             2.4516e-02,  2.8071e-02],
           [-9.3024e-01, -9.2436e-01, -8.3248e-01,  ..., -1.3988e-01,
             2.3705e-02,  2.4105e-02],
           [-7.9853e-01, -7.8901e-01, -7.0704e-01,  ..., -2.6335e-01,
            -1.8207e-01, -1.7858e-01],
 

In [5]:
b2r(a)

tensor([[[[[ 2.9661e-02,  3.1020e-02, -6.3282e-02,  ..., -4.1733e-02,
            -8.0748e-02, -8.6301e-02],
           [ 4.1666e-02,  3.8430e-02, -5.1725e-02,  ..., -3.0384e-02,
            -6.7830e-02, -8.1591e-02],
           [ 1.1521e-01,  1.1283e-01,  2.7561e-04,  ..., -1.8878e-01,
            -2.3380e-01, -2.3598e-01],
           ...,
           [-2.5038e+00, -2.5053e+00, -2.1959e+00,  ...,  8.0589e-01,
             7.4693e-01,  7.3153e-01],
           [-2.8181e+00, -2.8187e+00, -2.4807e+00,  ...,  9.5511e-01,
             8.4805e-01,  8.4830e-01],
           [-2.8126e+00, -2.8126e+00, -2.4764e+00,  ...,  9.5974e-01,
             8.5329e-01,  8.4924e-01]],

          [[-9.6907e-01, -9.6898e-01, -8.7984e-01,  ..., -1.8552e-01,
            -2.6158e-02, -2.3357e-02],
           [-9.6332e-01, -9.6223e-01, -8.7533e-01,  ..., -1.7356e-01,
            -1.9620e-02, -9.3877e-03],
           [-8.2849e-01, -8.3333e-01, -7.5358e-01,  ..., -3.1028e-01,
            -2.2258e-01, -2.2462e-01],
 